In [2]:
from jupyter_dash import JupyterDash

# import dash
import dash_leaflet as dl

from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
# from pymongo import MongoClient
import matplotlib.pyplot as plt
from bson.json_util import dumps
import re #needed for the regex pattern matching

import base64 #need for images

from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#username = "aacuser"
#password = "simple"
db = AnimalShelter()

#create the default dataframe
df = pd.DataFrame.from_records(db.read({}))
#df = pd.DataFrame.from_records(db.read({'animal_type':'Dog'}))

df.drop(columns=['_id'],inplace=True)
# df.drop(columns=['_id'],inplace=True)
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Company logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Anchor tag for heading
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('Matthew Cochrane SNHU CS-340 Dashboard'))),
    html.Hr(),

    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#        editable=True,
        editable=False,
        row_selectable="single", #allow a row to be selected
#        selected_rows=[],
        row_deletable=False,
        column_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        sort_mode="multi",
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page
        

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
   
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
 
    if filter_type == 'All':
        df = pd.DataFrame.from_records(db.read({}))
    
    elif filter_type == "Water":
        df = pd.DataFrame.from_records(db.read(
            {"$and":[
                {"$or":[
                {'breed':{"$regex":'Newfoundland'}},
                {'breed':{"$regex":'Labrador'}},
                {'breed':{"$regex":'Chesapeake'}}]},
                {'sex_upon_outcome':'Intact Female'},
                {"$and":[
                    {'age_upon_outcome_in_weeks':{"$gt":26}},
                    {'age_upon_outcome_in_weeks':{"$lt":156}}
                ]}
            ]}))
    
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(db.read(
            {"$and":[
                {"$or":[
                {'breed':{"$regex":'German'}},
                {'breed':{"$regex":'Malamute'}},
                {'breed':{"$regex":'Sheepdog'}},
                {'breed':{"$regex":'Husky'}},
                {'breed':{"$regex":'Rottweiler'}}]},
                {'sex_upon_outcome':'Intact Male'},
                {"$and":[
                    {'age_upon_outcome_in_weeks':{"$gt":26}},
                    {'age_upon_outcome_in_weeks':{"$lt":156}}
                ]}
            ]}))
    
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(db.read(
            {"$and":[
                {"$or":[
                {'breed':{"$regex":'Doberman'}},
                {'breed':{"$regex":'German'}},
                {'breed':{"$regex":'Golden'}},
                {'breed':{"$regex":'Bloodhound'}},
                {'breed':{"$regex":'Rottweiler'}}]},
                {'sex_upon_outcome':'Intact Male'},
                {"$and":[
                    {'age_upon_outcome_in_weeks':{"$gt":20}},
                    {'age_upon_outcome_in_weeks':{"$lt":300}}
                ]}
            ]}))
    else:
        raise Exception("Unknown filter")

    columns=[{"name": i,"id": i, "deletable": False, "selectable": True} for i in df.columns]
    df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')


# #change the color of a selected cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    }for i in selected_columns]

#call back for pie chart
#set to plot all of the data across all of the pages instead of the viewable data
#change to derived_viewport_data if other behavior is wanted


@app.callback(
    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed', title='Preferred Animals')
        )    
    ]

#call back for slecting a row and then plotting the geomarker

@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(virtualRows):
    #austin Texas is [30.75, -97.48]
    
    #create the views
    if not virtualRows: 
        markerArray = (30.75,-97.48)
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: #build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows])
        coordLat = float(dff['location_lat'].to_string().split()[1])
        coordLong = float(dff['location_long'].to_string().split()[1])
        markerArray = (coordLat, coordLong)
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
#        popUpParagraph = dff['name']
        popUpParagraph = df['name']

    #return the map with a child marker
    #marker is set to the values found in markerArray
    #map centers/moves to view the new marker instead of holding a fixed center
    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

app.run_server(debug=True)

initialized
Dash app running on http://127.0.0.1:10267/
